<a href="https://colab.research.google.com/github/VictoriaUsman/LLM-Engineering/blob/main/Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
from google.colab import userdata
userdata.get('HuggingFace')
from huggingface_hub import login
login(token=userdata.get('HuggingFace'))
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import gc


In [2]:
!pip install -q --upgrade  bitsandbytes accelerate

In [3]:
login(token=userdata.get('HuggingFace'), add_to_git_credential=True)

In [4]:
Llama = ""

In [5]:
message = [{
    "role":"user",
    "content":"Tell me something funny"}]

In [6]:
qnt_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [7]:
tokenizer = AutoTokenizer.from_pretrained(Llama)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(
    message,return_tensors="pt").to("cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
inputs

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271, 128009, 128006,
            882, 128007,    271,  41551,    757,   2555,  15526, 128009]],
       device='cuda:0')

In [9]:
model = AutoModelForCausalLM.from_pretrained(Llama, device_map = "auto", quantization_config = qnt_cfg)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [10]:
memory = model.get_memory_footprint()/1e6

In [13]:
print(f"Memory footprint of the model: {memory:,.1f} MB")

Memory footprint of the model: 5,591.5 MB


In [15]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRM

In [22]:
outputs = model.generate(inputs, max_new_tokens= 80)
outputs[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
            25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
           220,   1627,  10263,    220,   2366,     19,    271, 128009, 128006,
           882, 128007,    271,  41551,    757,   2555,  15526, 128009, 128006,
         78191, 128007,    271,     32,    893,  15203,   1139,    264,   6875,
           323,   4691,    279,  95307,     11,    330,   5519,    499,    617,
           904,   6603,    389,  43856,  36247,    596,  12875,    323,   5124,
            81,   3029,     67,   5248,    596,   8415,   7673,   4815,    791,
         95307,  20592,     11,    330,   2181,  25562,    264,  29519,     11,
           719,    358,   2846,    539,   2771,    422,    433,    596,   1618,
           477,    539,   1210, 128009], device='cuda:0')

In [23]:
print(tokenizer.decode(outputs[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell me something funny<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A man walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?" 

The librarian replied, "It rings a bell, but I'm not sure if it's here or not."<|eot_id|>
